<img src="http://akhavanpour.ir/notebook/images/srttu.gif" alt="SRTTU" style="width: 150px;"/>

[![Azure Notebooks](https://notebooks.azure.com/launch.png)](https://notebooks.azure.com/import/gh/Alireza-Akhavan/class.vision)

<div style="text-align: right;direction:rtl;font-family:tahoma">

# استفاده از راهکار شبکه عصبی ژرف

In [1]:
import tensorflow as tf

#Start interactive session
sess = tf.InteractiveSession()

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


<div style="text-align: right;direction:rtl;font-family:tahoma">

### پارامترهای اولیه

<div style="text-align: right;direction:rtl;font-family:tahoma">
ایجاد پارامترهای عمومی برای مدل:
</div>

In [4]:
width = 28 # width of the image in pixels 
height = 28 # height of the image in pixels
flat = width * height # number of pixels in one image 
class_output = 10 # number of possible classifications for the problem

<div style="text-align: right;direction:rtl;font-family:tahoma">
### ورودی  و خروجی

<div style="text-align: right;direction:rtl;font-family:tahoma">
ایجاد place holder برای ورودی ها و خروجی ها
</div>

In [5]:
x  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

<div style="text-align: right;direction:rtl;font-family:tahoma">
### وزن ها

<div style="text-align: right;direction:rtl;font-family:tahoma">
تعریف تابع برای تولید وزنها:
</div>

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

<div style="text-align: right;direction:rtl;font-family:tahoma">
### بایاس

<div style="text-align: right;direction:rtl;font-family:tahoma">
تعریف تابع برای تولید بایاس:
<hr>
باتوجه به ویژگی ReLU، نورون‌ها در طول آموزش بسیار حساس هستند.
برای جلوگیری از نورون‌های مرده، مقدار دهی اولیه با مقادیر مثبت کوچک انجام شده است.
</div>

In [7]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

<div style="text-align: right;direction:rtl;font-family:tahoma">
### لایه های کانولوشنالی

<div style="text-align: right;direction:rtl;font-family:tahoma">
تعریف یک تابع برای ایجاد لایه های کانولوشنالی
</div>


<img src="lecture_images/stride.png" alt="HTML5 Icon" style="width:600px;"> 


In [8]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

<div style="text-align: right;direction:rtl;font-family:tahoma">
### ادغام یا Max Pooling

<div style="text-align: right;direction:rtl;font-family:tahoma">
تعریف تابع برای انجام عمل max pooling یا ادغام بیشینه.
<ul>
<li>
__ اندازه Kernel: __ 
<br>
2x2  (اگر تصویر یک ماتریس 2x2 باشد، آنگاه در خروجی یک پیکسل نتیجه میدهد)
</li>
<li>
__Strides: __ 
<br>
گام های حرکت. در این مورد کرنل 2 پیکسل در هر حرکت جا به جا میشود، با توجه به اندازه ی کرنل، ادغام بدون همپوشانی است.
</li>
</ul>

In [9]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

<div style="text-align: right;direction:rtl;font-family:tahoma">
### اولین لایه ی کانولوشنالی (convolutional) و ادغام بیشینه ( max-pooling )

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### وزنها و بایاس

<div style="text-align: right;direction:rtl;font-family:tahoma">
    <ul>
        <li>
        اندازه فیلتر یا کرنل: 5x5 
        </li>
        <li>
        تعداد کانال ورودی: 1 (سیاه و سفید)
        </li>
        <li>
        تعداد feature mapها: 32 
        </li>
    </ul>
</div>

In [10]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32]) # need 32 biases for 32 outputs

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### تبدیل تصاویر  مجموعه داده  به تنسور

<div style="text-align: right;direction:rtl;font-family:tahoma">
28 پیکسل در 28 پیکسل و  1 کانال (سیاه سفید)
<br>
</div>

In [11]:
x_image = tf.reshape(x, [-1,28,28,1])  

<div style="text-align: right;direction:rtl;font-family:tahoma">
####  کانوالو کردن تصویر با ماتریس وزن ها و افزودن بایاس

In [12]:
convolve1= conv2d(x_image, W_conv1) + b_conv1

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### اعمال تابع فعالیت ReLU


<img src="lecture_images/relu.png" alt="HTML5 Icon" style="width:600px;"> 
<div style="text-align:center">
<div style="direction:rtl;font-family:tahoma">
واحد یکسو ساز خطی (ReLU)</div>
</div>

<div style="text-align: right;direction:rtl;font-family:tahoma">
یک تابع غیر خطی که معمولا برای مسائل طبقه بندی استفاده میشود (rectified linear unit)
</div>

In [13]:
h_conv1 = tf.nn.relu(convolve1)

In [14]:
h_conv1

<tf.Tensor 'Relu:0' shape=(?, 28, 28, 32) dtype=float32>

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### اعمال max pooling

<div style="text-align: right;direction:rtl;font-family:tahoma">
این تابع را قبلا تعریف کرده ایم.
</div>

In [17]:
h_pool1 = max_pool_2x2(h_conv1)

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### لایه ی نخست به اتمام رسید


In [18]:
layer1= h_pool1

<div style="text-align: right;direction:rtl;font-family:tahoma">
### دومین لایه ی کانولوشنالی (convolutional) و ادغام بیشینه ( max-pooling )

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### وزنها و بایاس

<div style="text-align: right;direction:rtl;font-family:tahoma">
    <ul>
        <li>
        اندازه فیلتر یا کرنل: 5x5 
        (25 پیکسل)
        </li>
        <li>
         تعداد کانال ورودی: 32 (خروجی لایه ی نخست)        </li>
        <li>
        تعداد feature mapها: 64 
        </li>
    </ul>
</div>

In [19]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64]) #need 64 biases for 64 outputs

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### تبدیل تصاویر  مجموعه داده  به تنسور

<div style="text-align: right;direction:rtl;font-family:tahoma">
قبلا در لایه ی نخست تبدیل شده...<br>
</div>

<div style="text-align: right;direction:rtl;font-family:tahoma">
####  کانوالو کردن تصویر با ماتریس وزن ها و افزودن بایاس

In [25]:
convolve2= conv2d(layer1, W_conv2) + b_conv2

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### اعمال تابع فعالیت ReLU

In [26]:
h_conv2 = tf.nn.relu(convolve2)

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### اعمال max pooling

In [27]:
h_pool2 = max_pool_2x2(h_conv2)

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### لایه ی دوم به اتمام رسید


In [28]:
layer2= h_pool2

<div style="text-align: right;direction:rtl;font-family:tahoma">
### لایه ی سوم

<div style="text-align: right;direction:rtl;font-family:tahoma">
این لایه تمام-متصل یا fully connected است.
</div>

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### وزنها و بایاس

<div style="text-align: right;direction:rtl;font-family:tahoma">
از لایه ی قبل 64 عدد feature map با سایز 7*7 داشتیم.
</div>

In [29]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### مسطح کردن خروجی لایه دوم

In [30]:
layer2_matrix = tf.reshape(layer2, [-1, 7*7*64])

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### ضرب ماتریسی (اعمال وزنها و بایاس)

In [31]:
matmul_fc1=tf.matmul(layer2_matrix, W_fc1) + b_fc1

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### اعمال تابع فعالیت ReLU

In [32]:
h_fc1 = tf.nn.relu(matmul_fc1)

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### لایه ی سوم به اتمام رسید


In [33]:
layer3= h_fc1

<div style="text-align: right;direction:rtl;font-family:tahoma">
### فاز اختیاری برای کاهش Over-fitting خواهد - Dropout

<div style="text-align: right;direction:rtl;font-family:tahoma">
    برای شبکه های عصبی بسیار بزرگ بسیار مفید است. در این مرحله برخی از ویژگی ها فراموش میشوند.
</div>


<img src="lecture_images/dropout.png" alt="HTML5 Icon" style="width:250px;"> 
<div style="text-align:center">
<div style="direction:rtl;font-family:tahoma">
Dropout</div>
</div>


In [34]:
keep_prob = tf.placeholder(tf.float32)
layer3_drop = tf.nn.dropout(layer3, keep_prob)

<div style="text-align: right;direction:rtl;font-family:tahoma">
### لایه ی نهایی

<div style="text-align: right;direction:rtl;font-family:tahoma">
یک لایه ی تمام متصل Softmax
</div>

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### وزنها و بایاس

<div style="text-align: right;direction:rtl;font-family:tahoma">
کانال ورودی: 1024 (نورونها از لایه 3)؛ تعداد خروجی:10
</div>

In [35]:
W_fc2 = weight_variable([1024, 10]) #1024 neurons
b_fc2 = bias_variable([10]) # 10 possibilities for digits [0,1,2,3,4,5,6,7,8,9]

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### ضرب ماتریسی (اعمال وزنها و بایاس)

In [36]:
matmul_fc2=tf.matmul(layer3_drop, W_fc2) + b_fc2

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### اعمال تابع فعال Softmax

In [37]:
y_conv= tf.nn.softmax(matmul_fc2)

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### لایه ی آخر به اتمام رسید


In [38]:
layer4= y_conv

<div style="text-align: right;direction:rtl;font-family:tahoma">
# خلاصه ای از شبکه عصبی کانولوشن عمیق طراحی شده

#### 0) Input - MNIST dataset
#### 1) Convolutional and Max-Pooling
#### 2) Convolutional and Max-Pooling
#### 3) Fully Connected Layer
#### 4) Processing - Dropout
#### 5) Readout layer - Fully Connected
#### 6) Outputs - Classified digits

<div style="text-align: right;direction:rtl;font-family:tahoma">
# تعریف توابع و آموزش مدل

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### تعریف معیار خطا یا loss function

In [39]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(layer4), reduction_indices=[1])
cost = tf.reduce_mean(cross_entropy)

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### تعریف بهینه ساز

In [40]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### پیش بینی شبکه

In [41]:
correct_prediction = tf.equal(tf.argmax(layer4,1), tf.argmax(y_,1))

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### دقت شبکه

In [42]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### مقداردهی اولیه ی متغیرها

In [43]:
sess.run(tf.global_variables_initializer())

<div style="text-align: right;direction:rtl;font-family:tahoma">
#### آموزش

In [39]:
for i in range(1101):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, float(train_accuracy)))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})


step 0, training accuracy 0.06
step 100, training accuracy 0.82
step 200, training accuracy 0.94
step 300, training accuracy 0.94
step 400, training accuracy 0.9
step 500, training accuracy 0.88
step 600, training accuracy 0.88
step 700, training accuracy 0.92
step 800, training accuracy 1
step 900, training accuracy 0.96
step 1000, training accuracy 0.94
step 1100, training accuracy 0.98


<div style="text-align: right;direction:rtl;font-family:tahoma">
# ارزیابی مدل

In [46]:
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9654


In [47]:
sess.close() #finish the session

<div class="alert alert-success alertsuccess" style="margin-top: 20px;text-align: right;direction:rtl;font-family:tahoma">
<font size = 3><strong>
با تشکر از توجه شما
</strong></font>


<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma"> دانشگاه تربیت دبیر شهید رجایی<br>مباحث ویژه 2 - یادگیری عمیق پیشرفته<br>علیرضا اخوان پور<br>97-98<br>
</div>
<a href="https://www.srttu.edu/">SRTTU.edu</a> - <a href="http://class.vision">Class.Vision</a> - <a href="http://AkhavanPour.ir">AkhavanPour.ir</a>
</div>

<hr>
<div style="text-align: right;direction:rtl;font-family:tahoma">
منابع:
</div>
https://bigdatauniversity.com/courses/deep-learning-tensorflow/
<br>
http://tensorflow.org/
<br>
https://gotocon.com/dl/goto-london-2016/slides/MartinGorner_TensorflowAndDeepLearningWithoutAPhD.pdf
<br>
http://sebastianruder.com/optimizing-gradient-descent/index.html#batchgradientdescent
<br>
http://yann.lecun.com/exdb/mnist/  